In [28]:
import numpy as np
import pandas as pd
import re
import string
import csv

### NEED TO EXPLAIN WHAT THIS CODE IS DOING
***SPACY - REMOVING ANY UNKNOWN PUNCTUATION, BUT KEEPING EVERYTHING ELSE IN, INCLUDING SPACES/WEIRD FORMATTING***

In [3]:
unprocessed_transcripts = pd.read_csv('processed_data/transcripts.csv').drop(['Unnamed: 0'],axis = 1)
display(unprocessed_transcripts)

,Text,Coherence
0,"3,000 years have I been fighting. Every mornin...",3
1,"“Dad, you 're on TV again !” I heard Eric 's v...",4
2,"When Tyler entered the ward, his daughter Vale...",4
3,His body was failing. He had taken care of it ...,4
4,"I saw the button. It was simple, red, no words...",5
...,...,...
1534,The human body is an amazing thing. It is able...,3
1535,"Once upon a time, a young woman broke her vase...",4
1536,"Once upon a time, two beautiful diamonds and a...",3
1537,Lora and her team had been searching for the l...,5


In [4]:
regex_sub = {
    re.compile(r'\. ⁇ '): ' ',
    re.compile(r' ⁇ '): '',
    re.compile(r'([^.?!;,])( )([^iI]?\'[a-z])'): r'\1\3',
    re.compile(r'([a-z])(-)([a-z])'): r'\1 \3',
    re.compile(r'\\\''): r'\'',
    re.compile(fr'(?![A-Za-z0-9\s“”’…{re.escape(string.punctuation)}]).'): '',
    re.compile(r'[\#\*\+\^\{\}\|~]'): '',
    re.compile(r'(,[\'"“”])'): r'\1 ',
    re.compile(r'<.*?>'): '',
    re.compile(r' +|\n'): ' ',
    re.compile(r' s '): '\'s ',
    re.compile(r' t '): '\'t '
}

processed_transcripts = unprocessed_transcripts.copy()
for i in unprocessed_transcripts.index:
    story_string = unprocessed_transcripts['Text'].iloc[i]
    for pattern, substitution in regex_sub.items():
        story_string = pattern.sub(substitution, story_string)
    processed_transcripts.loc[i,'Text'] = story_string

display(processed_transcripts)

,Text,Coherence
0,"3,000 years have I been fighting. Every mornin...",3
1,"“Dad, you're on TV again !” I heard Eric's voi...",4
2,"When Tyler entered the ward, his daughter Vale...",4
3,His body was failing. He had taken care of it ...,4
4,"I saw the button. It was simple, red, no words...",5
...,...,...
1534,The human body is an amazing thing. It is able...,3
1535,"Once upon a time, a young woman broke her vase...",4
1536,"Once upon a time, two beautiful diamonds and a...",3
1537,Lora and her team had been searching for the l...,5


In [5]:
processed_transcripts['Text'].iloc[1413] # 1413
# TODO: remove anything between <>
processed_transcripts['Text'].iloc[584]

". I hadn't seen it since my first visit. It was just around the corner, and my arrival had beenn't terrible. I pushed my way through the fog, and saw that I had fallen asleep at the fountain of youth, lying there with the sponge for a simple evening bath. I remembered seeing a man in a wadded up suit before I noticed that I had fallen asleep at a young age. It had been a regular occurrence for me. I was four at the time, and it still did look like a dream. My grandmother had taken me to a circus and watched the man push the sponge into my lollipop and drink the sweet liquid from the fountain. It was probably a hallucination. I had never been much to the strange place, though I could definitely remember it a bit. Grandma helped me make a manicoseze blanket, which I was aware of, and let me sit on the toilet for five hours while she worked. I didn't have any balls or all that, but some people thought she would make me some kind of jellyfish or something, and with a big ol'elephant hat. 

NEED TO ADD/FIX THE EXAMPLES

In [6]:
processed_transcripts.to_csv('processed_data/transcripts_spacy_formatted.csv')

# ***NEED TO EXPLAIN THIS TOO!!! LOCAL COHERENCE FROM HOFFMAN CODE***

In [7]:
coherence_processed_transcripts = processed_transcripts.copy()
for i in processed_transcripts.index:
    story_string = processed_transcripts['Text'].iloc[i]
    story_string_replaced = re.sub(r'[^A-Za-z0-9\s]', '', story_string)
    story_string_replaced = re.sub(r' +|\n', ' ', story_string_replaced)
    coherence_processed_transcripts.loc[i,'Text'] = story_string_replaced

display(coherence_processed_transcripts)

,Text,Coherence
0,3000 years have I been fighting Every morning ...,3
1,Dad youre on TV again I heard Erics voice from...,4
2,When Tyler entered the ward his daughter Valer...,4
3,His body was failing He had taken care of it v...,4
4,I saw the button It was simple red no words on...,5
...,...,...
1534,The human body is an amazing thing It is able ...,3
1535,Once upon a time a young woman broke her vase ...,4
1536,Once upon a time two beautiful diamonds and a ...,3
1537,Lora and her team had been searching for the l...,5


In [8]:
all_data_frames = []
for i in coherence_processed_transcripts.index:
    story = coherence_processed_transcripts['Text'].iloc[i]
    story_list = []
    word = story.split(' ')
    story_list.extend(word)
    story_list = [item for item in story_list if item != '' '']
    response_data_frame = pd.DataFrame({
                                        'study_id': i,
                                        'group': 'transcripts',
                                        'prompt': 'retell',
                                        'response': story_list
                                        })
    all_data_frames.append(response_data_frame)

concatenated_transcripts = pd.concat(all_data_frames, axis = 0)

display(concatenated_transcripts)

,study_id,group,prompt,response
0,0,transcripts,retell,3000
1,0,transcripts,retell,years
2,0,transcripts,retell,have
3,0,transcripts,retell,I
4,0,transcripts,retell,been
...,...,...,...,...
85,1538,transcripts,retell,the
86,1538,transcripts,retell,cause
87,1538,transcripts,retell,of
88,1538,transcripts,retell,the


In [9]:
concatenated_transcripts.to_csv('processed_data/transcripts_hoffman_coherence_formatted.txt', sep='\t', index = False, header = False)

# ***NEED TO EXPLAIN THIS TOO!!! SPLITTING BY SENTENCE***

In [34]:
story_lists = []

for i in processed_transcripts.index:
    story_string = processed_transcripts['Text'].iloc[i]
    story_string_split = re.split(r'[;\.?!“”…"]+' + '|' + r'[\s]- ' + '|' + r'\' ', story_string)
    story_string_split = [item for item in story_string_split if item != '' '']
    story_lists.append(story_string_split)

print(story_lists[0])

['3,000 years have I been fighting', ' Every morning, the raccoons scratch at my eyes', ' Every evening, the skunks spray me while the opossums chew at my feet', ' I have never had any tools', ' I have only my hands', ' I don’t remember the place I came from before this', ' All I remember is the daily fight between me and these animals', ' No matter how many times I kill them, they come back the next day', ' No matter how many times I’ve ripped them limb from limb, they are here for their appointment the next day just as eager to tear me apart', ' They want my body to be destroyed beyond recognition, and most days they succeed', ' When I wake up in the morning, all my wounds from the day before are gone', ' Not even a scratch on my little toe', ' Why do these animals want to hurt me so bad', ' What have I done to deserve this fate', ' All I know anymore is fighting', ' The struggle', ' But we aren’t struggling for a purpose, we’re just here', ' No one else has ever peered in to our for

In [33]:
with open("processed_data/sentences.csv", "w") as f:
    wr = csv.writer(f)
    wr.writerows(story_lists)